In [1]:
# Using PyTorch for NN
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import pandas as pd
import numpy as np
import inputparser

In [3]:
input_vector, y_test = inputparser.make_input()

In [4]:
# x-test, y-test are 225 data points, remove first element for verification
x_verification = []
x_verification.append(input_vector[0])
y_verification = y_test[0]

x_input = input_vector[1:]
y_input = []
for i in y_test[1:]:
    y_input.append(i)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Input data
X = torch.tensor(x_input.tolist())
Y = torch.tensor(y_input)

# Split data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2,random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define the model
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
input_size = len(x_input[0])
model = NeuralNet(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

# Training loop
best_val_loss = float('inf')
patience = 10
counter = 0
for epoch in range(1000):
    # Forward pass
    outputs = model(torch.tensor(X_train, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), Y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation loss
    with torch.no_grad():
        val_outputs = model(torch.tensor(X_val, dtype=torch.float32))
        val_loss = criterion(val_outputs.squeeze(), Y_val)
        
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping on epoch {epoch}.')
            break

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Test
input_data = torch.tensor(np.array(x_verification))
input_data = scaler.transform(input_data) # Normalize
predicted_output = model(torch.tensor(input_data, dtype=torch.float32))
print("Predicted output:", predicted_output.item())
print("True output:", y_verification)

Epoch [10/1000], Loss: 6.5634, Val Loss: 60.0878
Epoch [20/1000], Loss: 0.3401, Val Loss: 1.1783
Early stopping on epoch 21.
Predicted output: -1.775876760482788
True output: -1.9075102999999842


In [6]:
from sklearn.metrics import mean_squared_error
def calculate_mse(model, X, Y):
    outputs = model(torch.tensor(X, dtype=torch.float32))
    loss = mean_squared_error(outputs.squeeze().detach().numpy(), Y)
    return loss

train_mse = calculate_mse(model, X_train, Y_train)
print("Training MSE:", train_mse)

val_mse = calculate_mse(model, X_val, Y_val)
print("Validation MSE:", val_mse)

Training MSE: 0.31373543
Validation MSE: 1.1251938
